In [201]:
import csv

from pymystem3 import Mystem
my = Mystem()

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize

from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

from sklearn.metrics import accuracy_score

import copy

(1 балл) Создание, разметка корпуса и объяснение того, почему этот текст подходит для оценки (какие моменты вы тут считаете трудными для автоматического посттеггинга и почему, в этом вам может помочь второй ридинг). Не забывайте, что разные теггеры могут использовать разные тегсеты: напишите комментарий о том, какой тегсет вы берёте для разметки и почему.





In [71]:
'''Сложности в тексте:
- имена собственные
- аббревиатуры (МГУ им. М.В.Ломоносова)
- соркащения (ср., и т.д.)
- слова через дефис (в марте-апреле, IT-технологии)
- составные предлоги и союзы (в связи с, как... так и)
- вводные слова (например, наконец, счастливым образом)
- наречия на основе словосочетаний (как минимум, как максимум)
- нестандартная пунктуация (')
- модальные предикаты, союзные слова, частица "ли" - все они распознаются парсерами как попало
'''

#Текст без разметки
with open('forum.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [158]:
'''Для разметки выбран тегсет UPOS плюс тег PARENTH для вводных слов. 
Выбираю его, так как он основан на общепринятых UD и адаптирован для русского материала.
Но тега для вводных слов там нет...'''

#Текст с ручной разметкой, формат TSV
table = []

with open('forum_table.txt', 'r', encoding='utf-8') as f:
    tsvreader = csv.reader(f, delimiter="\t")
    for row in tsvreader:
        table.append([row[0], row[1]])
    f.close()
    
print(table[:10])

[['Форум', 'NOUN'], ['«', 'PUNCT'], ['Оценка', 'NOUN'], ['методов', 'NOUN'], ['автоматического', 'ADJ'], ['анализа', 'NOUN'], ['текста', 'NOUN'], ['»', 'PUNCT'], ['стартовал', 'VERB'], ['в', 'ADP']]


(3 балла) Потом вам будет нужно взять три POS теггера для русского языка (udpipe, stanza, natasha, pymorphy, mystem, spacy, deeppavlov) и «прогнать» текст через каждый из них.

In [196]:
#Mystem
mystem_tags = []
mystem_an = my.analyze(text)

for word in mystem_an:
    if 'analysis' in word:
        if word['analysis'] == []:
            mystem_tags.append((word['text'], 'X'))
        else:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            mystem_tags.append([word['text'], pos])
            
print(mystem_tags[:10])

[['Форум', 'S'], ['Оценка', 'S'], ['методов', 'S'], ['автоматического', 'A'], ['анализа', 'S'], ['текста', 'S'], ['стартовал', 'V'], ['в', 'PR'], ['феврале', 'S'], ['года', 'S']]


In [101]:
#Pymorphy
pymorphy_tags = []

for token in simple_word_tokenize(text):
    pymorphy_an = morph.parse(token)
    pos = str(pymorphy_an[0].tag)
    pymorphy_tags.append([pymorphy_an[0].word, pos[:4]])
    
print(pymorphy_tags[:10])

[['форум', 'NOUN'], ['«', 'PNCT'], ['оценка', 'NOUN'], ['методов', 'NOUN'], ['автоматического', 'ADJF'], ['анализа', 'NOUN'], ['текста', 'NOUN'], ['»', 'PNCT'], ['стартовал', 'VERB'], ['в', 'PREP']]


In [102]:
#Natasha
natasha_tags = []

doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

for token in doc.tokens:
    natasha_tags.append([token.text, token.pos])

print(natasha_tags[:10])

[['Форум', 'NOUN'], ['«', 'PUNCT'], ['Оценка', 'NOUN'], ['методов', 'NOUN'], ['автоматического', 'ADJ'], ['анализа', 'NOUN'], ['текста', 'NOUN'], ['»', 'PUNCT'], ['стартовал', 'VERB'], ['в', 'ADP']]


(2 балла) Затем оценим accuracy для каждого теггера. Заметьте, что в разных системах имена тегов и части речи могут отличаться, – вам надо будет свести это всё к единому стандарту с помощью какой-то функции-конвертера и сравнить с вашим размеченным руками эталоном - тоже с помощью какого-то кода или функции.

In [197]:
mystem_dict = {
    'S': 'NOUN',
    'A': 'ADJ',
    'V': 'VERB',
    'PR': 'ADP',
    'CONJ': 'CCONJ',
    'SPRO': 'PRON',
    'ADVPRO': 'ADV',
    'APRO': 'DET',
    'X': 'X',
    'PART': 'PART',
    'ANUM': 'ADJ',
    'ADV': 'ADV'
}

new_mystem_tags = []

for tag in mystem_tags:
    new_mystem_tags.append([tag[0], mystem_dict[tag[1]]])
    
print(new_mystem_tags[:10])

[['Форум', 'NOUN'], ['Оценка', 'NOUN'], ['методов', 'NOUN'], ['автоматического', 'ADJ'], ['анализа', 'NOUN'], ['текста', 'NOUN'], ['стартовал', 'VERB'], ['в', 'ADP'], ['феврале', 'NOUN'], ['года', 'NOUN']]


In [224]:
pymorphy_dict = {
    'NOUN': 'NOUN',
    'VERB': 'VERB',
    'PNCT': 'PUNCT',
    'INFN': 'VERB',
    'ADJF': 'ADJ',
    'PREP': 'ADP',
    'CONJ': 'CCONJ',
    'LATN': 'X',
    'ADVB': 'ADV',
    'PRTF': 'VERB',
    'NUMB': 'NUM',
    'NPRO': 'DET',
    'PRED': 'VERB',
    'PRCL': 'PART',
    'ADJS': 'ADJ'
}

new_pymorphy_tags = []

for tag in pymorphy_tags:
    new_pymorphy_tags.append([tag[0], pymorphy_dict[tag[1]]])
    
print(new_pymorphy_tags[:10])

[['форум', 'NOUN'], ['«', 'PUNCT'], ['оценка', 'NOUN'], ['методов', 'NOUN'], ['автоматического', 'ADJ'], ['анализа', 'NOUN'], ['текста', 'NOUN'], ['»', 'PUNCT'], ['стартовал', 'VERB'], ['в', 'ADP']]


In [199]:
'''Чтобы измерить accurcay, придётся справиться с тем, что кол-во токенов получилось разным. 
Mystem, например, игнорирует всю пунктуацию.'''
print(len(new_mystem_tags), len(new_pymorphy_tags), len(natasha_tags), len(table))

225 283 281 266


In [198]:
y_true = []

for token in table:
    y_true.append(token[1])

In [235]:
y_mystem = []

mystem_copy = copy.deepcopy(new_mystem_tags)

for true_token in table:
    for pred_token in mystem_copy:
        if true_token[0] == pred_token[0]:
            y_mystem.append(pred_token[1])
            mystem_copy.remove(pred_token)
            break
        elif pred_token == mystem_copy[-2]: #элемент [-1] - это '.'
            y_mystem.append('N/A')
            break

In [236]:
accuracy_score(y_true, y_mystem)

0.6954887218045113

In [230]:
y_pymorphy = []

pymorphy_copy = copy.deepcopy(new_pymorphy_tags)

for true_token in table:
    for pred_token in pymorphy_copy:
        if true_token[0].lower() == pred_token[0]:
            y_pymorphy.append(pred_token[1])
            pymorphy_copy.remove(pred_token)
            break
        elif pred_token == pymorphy_copy[-2]: 
            y_pymorphy.append('N/A')
            break

In [233]:
accuracy_score(y_true, y_pymorphy)

0.8345864661654135

In [237]:
y_natasha = []

natasha_copy = copy.deepcopy(natasha_tags)

for true_token in table:
    for pred_token in natasha_copy:
        if true_token[0] == pred_token[0]:
            y_natasha.append(pred_token[1])
            natasha_copy.remove(pred_token)
            break
        elif pred_token == natasha_copy[-2]: 
            y_natasha.append('N/A')
            break

In [238]:
accuracy_score(y_true, y_natasha)

0.8308270676691729

(2 балла) Дальше вам нужно взять лучший теггер для русского языка и с его помощью написать функцию (chunker), которая выделяет из размеченного текста 3 типа n-грамм, соответствующих какому-то шаблону (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд)

In [253]:
#у пайморфи и наташи получился примерно одинаковый результат. возьму наташу, чтобы не путаться в pos-тегах

def chuncker(text):
    natasha_tags = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        natasha_tags.append([token.text, token.pos])

    adp_noun = []
    for i in range(len(natasha_tags)-1):
        if natasha_tags[i][1] == 'ADP':
            if natasha_tags[i+1][1] == 'NOUN':
                adp_noun.append(natasha_tags[i][0] + ' ' + natasha_tags[i+1][0])
                
    adj_noun = []
    for i in range(len(natasha_tags)-1):
        if natasha_tags[i][1] == 'ADJ':
            if natasha_tags[i+1][1] == 'NOUN':
                adj_noun.append(natasha_tags[i][0] + ' ' + natasha_tags[i+1][0])
                
    before_dot = []
    for i in range(len(natasha_tags)-1):
        if natasha_tags[i][0] == '.':
            before_dot.append(natasha_tags[i-1][0] + natasha_tags[i][0])
            
    return adp_noun, adj_noun, before_dot

In [254]:
chuncker(text)

(['в феврале',
  'в марте-апреле',
  'на конференции',
  'на существование',
  'для построения',
  'в мире',
  'В связи',
  'в названии',
  'при анализе'],
 ['автоматического анализа',
  '2010 года',
  'первого года',
  'морфологические парсеры',
  'русского языка',
  'Тестовый запуск',
  'очную встречу',
  'постоянными участниками',
  'русского языка',
  'новые процессоры',
  'лингвистические ресурсы',
  'лингвистических систем',
  'разных этапах',
  'большой популярностью',
  'некоммерческие семинары',
  'сравнительной оценке',
  'русскоязычных ресурсов',
  'Счастливым образом',
  '2010 года',
  'образовательную составляющую',
  'финального отчета',
  'активное участие',
  'прикладной лингвистики',
  'филологического факультета',
  'слабые стороны',
  'внутренней документации',
  'морфологический парсер',
  'текстового слова'],
 ['языка.', '».', 'ср.', 'др.', 'ресурсов.', 'им.', 'М.', 'В.', 'т.', 'д.'])

(2-3 балла) В предыдущем дз многие из вас справедливо заметили, что если бы мы могли класть в словарь не только отдельные слова, но и словосочетания, то программа работала бы лучше. Предложите 3 шаблона (слово + POS-тег / POS-тег + POS-тег) запись которых в словарь, по вашему мнению, улучшила бы качество работы программы из предыдущей домашки. Балл за объяснение того, почему именно эти группы вы взяли, балл за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без (это бонусный одиннадцатый балл).

1. ADJ + цена 
2. ADJ + качество
3. не + VERB

1 и 2 помогают отличить "низкую цену" и "высокое качество" от "высокой цены" и "низкого качества" - без сущ. значение прил. не даст нам определить окраску отзыва.
3 поможет отличить "не понравился", "не оценил", "не стоит" от "не подвёл", "не разочаровал" и т.п.